In [1]:
import os, glob
import datetime
from requests import get
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In [2]:
## COLLECT DATA FROM MINISTERY OF HEALTH

web_url = "https://www.mohfw.gov.in"
file_store_loc = "/Volumes/Lab/PROJECTS/COVID-19-AnalyticsHub/India/data/mhw_ind_covid_daily_reports/"
def ind_mh_web_scrap(url, data_dir):
    #df = pd.read_html(url, header=0)[1].iloc[:,1:]
    df = pd.read_html(url)[7].iloc[:-1,1:] #Read covid-19 india 
    df.columns = ['State/UT', 'Confirmed_IN','Confirmed_FN','Recovered', 'Death']
    df.loc[(df['State/UT'].str.contains("Total")),'State/UT'] = "India"
    df = df.select_dtypes(include=['object']).apply(lambda x: x.str.replace('#',''))
    df['report_date'] = pd.Timestamp.now().normalize()
    df['report_date'] = pd.to_datetime(df['report_date']).dt.date
    df['State/UT']
    file_name = datetime.datetime.now().strftime("%Y-%m-%d")
    df.to_csv(data_dir+file_name+'.csv', index=False)
    return df

    
if __name__ == "__main__":
    daily_report = ind_mh_web_scrap(web_url,file_store_loc)

In [25]:
def load_reports(file_loc):
    geo_loc = '/Volumes/Lab/PROJECTS/COVID-19-AnalyticsHub/India/data/state_geo_table.csv'
    ind_state_geo = pd.read_csv(geo_loc, index_col=0).reset_index()
    files = sorted([file for file in os.listdir(file_store_loc)])[2:]
    reports_df = pd.concat((pd.read_csv(file_loc+ file) for file in files),ignore_index=False) 
    reports_df['report_date'] = pd.to_datetime(reports_df['report_date']).dt.date
    reports_df['State/UT'] = reports_df['State/UT'].str.replace('Orissa', 'Odisha')
    reports_df['Confirmed_ALL'] = [x+y for x,y in zip(reports_df.Confirmed_IN, reports_df.Confirmed_FN)]
    reports_df = reports_df.merge(ind_state_geo, on='State/UT', how = 'left')
    reports_df = reports_df.sort_values(by='Confirmed_ALL',ascending=False).groupby(['report_date', 'State/UT']).first().reset_index()
    reports_df_yest = reports_df.copy()
    reports_df_yest['report_date'] = pd.to_datetime(reports_df_yest['report_date']).dt.date + pd.to_timedelta(np.ceil(1), unit="D")
    return reports_df, reports_df_yest

In [26]:
df1, df2 = load_reports(file_store_loc)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 218 entries, 0 to 217
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   report_date    218 non-null    object 
 1   State/UT       218 non-null    object 
 2   Confirmed_IN   218 non-null    int64  
 3   Confirmed_FN   218 non-null    int64  
 4   Recovered      218 non-null    int64  
 5   Death          218 non-null    int64  
 6   Confirmed_ALL  218 non-null    int64  
 7   lat            216 non-null    float64
 8   lon            216 non-null    float64
dtypes: float64(2), int64(5), object(2)
memory usage: 15.5+ KB


In [3]:
daily_report['State/UT'].unique()

array(['Andaman and Nicobar Islands', 'Andhra Pradesh', 'Bihar',
       'Chandigarh', 'Chhattisgarh', 'Delhi', 'Goa', 'Gujarat', 'Haryana',
       'Himachal Pradesh', 'Jammu and Kashmir', 'Karnataka', 'Kerala',
       'Ladakh', 'Madhya Pradesh', 'Maharashtra', 'Manipur', 'Mizoram',
       'Odisha', 'Puducherry', 'Punjab', 'Rajasthan', 'Tamil Nadu',
       'Telengana', 'Uttarakhand', 'Uttar Pradesh', 'West Bengal',
       'India'], dtype=object)

In [4]:
#Static data India state loading
file_loc = '/Volumes/Lab/PROJECTS/COVID-19-AnalyticsHub/India/data/state_geo_table.csv'
ind_state_geo = pd.read_csv(file_loc, index_col=0).reset_index()

In [5]:
files = sorted([file for file in os.listdir(file_store_loc)])[2:]
reports_df = pd.concat((pd.read_csv(file_store_loc+ file) for file in files),ignore_index=False) 
reports_df['report_date'] = pd.to_datetime(reports_df['report_date']).dt.date
reports_df['State/UT'] = reports_df['State/UT'].str.replace('Orissa', 'Odisha')
reports_df['Confirmed_ALL'] = [x+y for x,y in zip(reports_df.Confirmed_IN, reports_df.Confirmed_FN)]

In [6]:
reports_df.head()

,State/UT,Confirmed_IN,Confirmed_FN,Recovered,Death,report_date,Confirmed_ALL
0,Andaman and Nicobar Islands,1,0,0,0,2020-03-26,1
1,Andhra Pradesh,11,0,1,0,2020-03-26,11
2,Bihar,6,0,0,1,2020-03-26,6
3,Chandigarh,7,0,0,0,2020-03-26,7
4,Chhattisgarh,6,0,0,0,2020-03-26,6


In [7]:
reports_df = reports_df.merge(ind_state_geo, on='State/UT', how = 'left')
reports_df = reports_df.sort_values(by='Confirmed_ALL',ascending=False).groupby(
    ['report_date', 'State/UT']).first().reset_index()
reports_df_yest = reports_df.copy()
reports_df_yest['report_date'] = pd.to_datetime(reports_df_yest['report_date']).dt.date \
                                + pd.to_timedelta(np.ceil(1), unit="D")


In [8]:
final_df = reports_df.merge(reports_df_yest.iloc[:,:-2], on=['report_date','State/UT'],
                   suffixes=('', '_yest'), how='left')
final_df.fillna(0, inplace=True)

In [9]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218 entries, 0 to 217
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   report_date         218 non-null    object 
 1   State/UT            218 non-null    object 
 2   Confirmed_IN        218 non-null    int64  
 3   Confirmed_FN        218 non-null    int64  
 4   Recovered           218 non-null    int64  
 5   Death               218 non-null    int64  
 6   Confirmed_ALL       218 non-null    int64  
 7   lat                 218 non-null    float64
 8   lon                 218 non-null    float64
 9   Confirmed_IN_yest   218 non-null    float64
 10  Confirmed_FN_yest   218 non-null    float64
 11  Recovered_yest      218 non-null    float64
 12  Death_yest          218 non-null    float64
 13  Confirmed_ALL_yest  218 non-null    float64
dtypes: float64(7), int64(5), object(2)
memory usage: 25.5+ KB


In [10]:
final_df['new_cases'] = final_df.Confirmed_ALL - final_df.Confirmed_ALL_yest
final_df['new_Deaths'] = final_df.Death - final_df.Death_yest
final_df['new_Recovd'] = final_df.Recovered - final_df.Recovered_yest
final_df['report_date'] = pd.to_datetime(final_df['report_date']).dt.date

In [11]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 218 entries, 0 to 217
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   report_date         218 non-null    object 
 1   State/UT            218 non-null    object 
 2   Confirmed_IN        218 non-null    int64  
 3   Confirmed_FN        218 non-null    int64  
 4   Recovered           218 non-null    int64  
 5   Death               218 non-null    int64  
 6   Confirmed_ALL       218 non-null    int64  
 7   lat                 218 non-null    float64
 8   lon                 218 non-null    float64
 9   Confirmed_IN_yest   218 non-null    float64
 10  Confirmed_FN_yest   218 non-null    float64
 11  Recovered_yest      218 non-null    float64
 12  Death_yest          218 non-null    float64
 13  Confirmed_ALL_yest  218 non-null    float64
 14  new_cases           218 non-null    float64
 15  new_Deaths          218 non-null    float64
 16  new_Reco

In [12]:
final_df.tail()

,report_date,State/UT,Confirmed_IN,Confirmed_FN,Recovered,Death,Confirmed_ALL,lat,lon,Confirmed_IN_yest,Confirmed_FN_yest,Recovered_yest,Death_yest,Confirmed_ALL_yest,new_cases,new_Deaths,new_Recovd
213,2020-03-27,Tamil Nadu,23,6,1,1,29,12.920385,79.150040,20.0,6.0,1.0,1.0,26.0,3.0,0.0,0.0
214,2020-03-27,Telengana,35,10,1,0,45,17.123184,79.208824,34.0,10.0,1.0,0.0,44.0,1.0,0.0,0.0
215,2020-03-27,Uttar Pradesh,40,1,11,0,41,27.599981,78.050003,40.0,1.0,11.0,0.0,41.0,0.0,0.0,0.0
216,2020-03-27,Uttarakhand,4,1,0,0,5,30.316496,78.032188,4.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0
217,2020-03-27,West Bengal,10,0,0,1,10,22.580391,88.329948,10.0,0.0,0.0,1.0,10.0,0.0,0.0,0.0


In [13]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk

In [14]:
es = Elasticsearch(['https://a149be5c55774db58b9b12c4741d5e95.asia-south1.gcp.elastic-cloud.com:9243'], 
                    http_auth=('elastic', 'hftyQ1VbGu9JJnbIFiEvDW0R'), timeout = 3000)
# delete index if exists
if es.indices.exists('india-covid19-db01'):
    es.indices.delete(index='india-covid19-db01')

settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
  },
  "mappings": {
    "properties": {
      "location": {
        "type": "geo_point"
      }
    }
  }
}

In [15]:
es.indices.create(index='india-covid19-db01', ignore=400, body=settings)
documents = final_df.to_dict(orient='records')
for i in range(0, len(documents)):
    documents[i].update({'location': {'lat':documents[i]['lat'], 'lon': documents[i]['lon']}})
bulk(es, documents, index = 'india-covid19-db01', raise_on_error = True)

(218, [])